In [ ]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED=False

In [ ]:
import os
import sys
sys.path.append("../")
import folderparser as fp
import folderscraper as fs
import interfacemetrics as intm
from interfacePlots import *
from config import cfg
LOGGERDEFINED = fp.openLog('interfacemetrics_viscsweep.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=True)

In [ ]:
EFOLDERS = [os.path.join(cfg.path.server, s) for s in ['HBnewtsweep', 'newtHBsweep', 'newtnewtsweep', 'HBHBsweep']]
EXPORTFOLDER = os.path.join(cfg.path.fig, 'viscositysweep', 'plots')

# Generating files

In [ ]:
import pandas as pd
import ncreate3d as nc
folder = r'\\cfs2e.nist.gov\642\NIST_Projects\Additive Manufacturing and Rheology\OpenFOAM\simulations\viscositysweep\HBHBsweep\nb347'
x = nc.ble+nc.frontWidth*nc.niw+8*nc.niw # cross-section taken 8 inner diameters behind the nozzle
# x = 1.4544
p = intm.posSlice(folder,x)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from descartes import PolygonPatch
import matplotlib.pyplot as plt

SCALE = 0.001;

yloc = nc.ncy-nc.niw # desired location of left edge of cross section
y = p['y']
z = p['z']
ymin = min(y)
y = [a-ymin+yloc for a in y]
vertices = list(zip(y,z)) # list of tuples

points = np.asarray(vertices) # numpy array of coordinate pairs

# turn xs into 3d shape
points3D = np.zeros((1,3))
for x in np.linspace(-4.824,4.824,50):
    for i in range(len(points)):
        if np.all(points3D==0):
            points3D = np.insert(points[i],0,x)
        else:
            points3D = np.vstack((points3D,np.insert(points[i],0,x)))

# Create triangles
from scipy.spatial import Delaunay
tri = Delaunay(points3D)
faces = tri.simplices

# Create  stl
from stl import mesh
xs = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        xs.vectors[i][j] = points3D[f[j], :]*SCALE
        
xs.save('inkLine.stl')
print('Done')

In [ ]:
help(xs.vectors)

In [ ]:
y = list(p['y'])
z = list(p['z'])
diffy = []
diffz = []
difffy = []
difffz = []
listt = []
count = 0
for i in range(len(y)-1):
    dy = y[i+1]-y[i]
    dz = z[i+1]-z[i]
    diffy.append(dy)
    diffz.append(dz)
    # if d < 0.0001:
    #     listt.append(i)
        
# for j in range(len(listt)):
#     if j not in listt:
#         count+=1
        
# print(count)
# print(len(y))

for i in range(len(diffz)-1):
    dy = diffy[i+1]-diffy[i]
    dz = diffz[i+1]-diffz[i]
    difffy.append(dy)
    difffz.append(dz)
    
# for i in range(len(difffz)):
#     if difffy[i]<0.000
    
# print(listt)
# print(len(np.unique(diffy)))
# print(len(np.unique(diffz)))
print(len(diffz))
# print(diffy)
# print(diffz)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import jpype
# import asposecells
# jpype.startJVM()
# from asposecells.api import *

# print(p[p['alpha']<.5])
# print(min(p['y']))
# print(min(p['z']))
y = p['y']
z = p['z']

fig, ax = plt.subplots()
ax.scatter(y, z)

ax.set(xlabel='y', ylabel='z',
       title='Cross-Section')
ax.grid()
plt.axis('scaled')
plt.show()

In [ ]:
import pandas as pd
o = []
for topfolder in [r'\\cfs2e.nist.gov\642\NIST_Projects\Additive Manufacturing and Rheology\OpenFOAM\simulations\viscositysweep\archive\mesh refinement']:
    for f in fp.caseFolders(topfolder):
        fs.populate(f)
        l = fp.legendUnique(f)
        if len(l)>0:
            o.append(l)
p = pd.DataFrame(o)
p.to_csv(r'\\cfs2e.nist.gov\642\NIST_Projects\Additive Manufacturing and Rheology\OpenFOAM\simulations\viscositysweep\archive\mesh refinement\legend_mesh.csv')

In [ ]:
import pandas as pd
o = []
for topfolder in EFOLDERS:
    for f in os.listdir(topfolder):
        l = fp.legendUnique(os.path.join(topfolder, f))
        if len(l)>0:
            o.append(l)
p = pd.DataFrame(o)
p.to_csv(os.path.join(cfg.fig, 'viscositysweep', 'legend_general.csv'))

In [ ]:
for topfolder in EFOLDERS:
    for folder in fp.caseFolders(topfolder):
        print(folder)
        print(type(folder))
        # intm.sumAndSteady(folder, True)

# Plots

In [ ]:
fullv = [10**i for i in range(-2, 6)]
for TOPFOLDER in EFOLDERS:
        xposition = 5
        for t in [2.5]:
            for s in [['arean', 1.0, 1.8]]:
                metricPlots(TOPFOLDER, EXPORTFOLDER, t, xposition, s[0], xvar='nuink', yvar='nusup', split=True, sigmalist=[0,40], overwrite=True, nuinklist=fullv, nusuplist=fullv, adjustBounds=False, tmin=s[1], tmax=s[2])

In [ ]:
fullv = [10**i for i in range(-2, 6)]
for TOPFOLDER in EFOLDERS:
        for sigma in [0, 40]:
            imsize=1
            steadyPlots(TOPFOLDER, imsize, EXPORTFOLDER, [sigma], xvar='nuink', yvar='nusup', overwrite=True, nuinklist=fullv, nusuplist=fullv, adjustBounds=False)

In [ ]:
for s in ['newtnewtsweep', 'controls']:
    TOPFOLDER = os.path.join(cfg.path.server, 'viscositysweep', s)
    xvar = 'nuink'
    yvar = 'nusup'
    fullv = [10**i for i in range(1, 6)]
    t = 2.5
    sigma = 0
    suplist = [10**3]
    for ta in ['y_py', 'x_uslicex', 'y_umag', 'y_uzslicey']:
        picPlots0(TOPFOLDER, EXPORTFOLDER, t, sigma, tag=ta, xvar=xvar, yvar=yvar, overwrite=False, nuinklist=fullv, nusuplist=suplist, adjustBounds=False)

In [ ]:
for s in ['newtnewtsweep']:
    TOPFOLDER = os.path.join(cfg.path.server, 'viscositysweep', s)
    xvar = 'nuink/nusup'
    yvar = '1/nusup'
    t = 1
    sigma = 0
    for ta in ['y_umag']:
        picPlots0(TOPFOLDER, EXPORTFOLDER, t, sigma, tag=ta, xvar=xvar, yvar=yvar, overwrite=True, adjustBounds=True)

In [ ]:
fullv = [10**i for i in range(-2, 6)]

xvar = 'nuink'
yvar = 'nusup'

for TOPFOLDER in [EFOLDERS[0]]:
    txtPlots0(TOPFOLDER, EXPORTFOLDER, xvar=xvar, yvar=yvar, overwrite=False)
    timePlots(TOPFOLDER, EXPORTFOLDER, xvar=xvar, yvar=yvar, overwrite=False)
    runtimePlots0(TOPFOLDER, EXPORTFOLDER, xvar=xvar, yvar=yvar, overwrite=False)
    
    xposition = 5
    for t in [2.5]:
        for s in [['arean', 0.7, 1.8], ['vertdispn', 0, 1], ['aspectratio', 0.6, 1.7], ['speeddecay', 0.5, 1.0]]:
            metricPlots(TOPFOLDER, EXPORTFOLDER, t, xposition, s[0], xvar=xvar, yvar=yvar, split=True,\
                        sigmalist=[0,40], overwrite=False, nuinklist=fullv, nusuplist=fullv, \
                        adjustBounds=False, tmin=s[1], tmax=s[2])
    for t in [0.5, 1, 2.5]:
        for sigma in [0, 40]:
            for ta in ['y_umag', 'y_viscy', 'y_uslicey', 'x_uslicex', 'x_viscx']:
                picPlots0(TOPFOLDER, EXPORTFOLDER, t, sigma, tag=ta, xvar=xvar, yvar=yvar, overwrite=False,\
                          nuinklist=fullv, nusuplist=fullv, adjustBounds=False)
    for t in [2.5]:
        XSPlots0(TOPFOLDER, EXPORTFOLDER, t, xposition, [0, 40], xvar=xvar, yvar=yvar, overwrite=False,\
                 nuinklist=fullv, nusuplist=fullv, adjustBounds=False)
    for sigma in [0, 40]:
        imsize=1.3
        steadyPlots(TOPFOLDER, imsize, EXPORTFOLDER, [sigma], xvar=xvar, yvar=yvar, overwrite=False, \
                    nuinklist=fullv, nusuplist=fullv, adjustBounds=False)

In [ ]:
fullv = [10**i for i in range(-2, 6)]

xvar = 'nuink'
yvar = 'nusup'

for TOPFOLDER in EFOLDERS:
    for t in range(0,26):
        t = round(t/10, 1)
        for sigma in [0, 40]:
            for ta in ['y_umag']:
                picPlots0(TOPFOLDER, EXPORTFOLDER, t, sigma, tag=ta, xvar=xvar, yvar=yvar, overwrite=False, nuinklist=fullv, nusuplist=fullv, adjustBounds=False, svg=False)

In [ ]:
for mode in [0,1]:
    flist = []
    for f in [533, 37, 45, 53]:
        flist.append(os.path.join(EFOLDERS[2], 'nb'+str(f)))
    fig = linePlots(flist, intmp.divfunc, 1, 2.25, mode)
    fig.savefig(os.path.join(EXPORTFOLDER, r'newtnewt_velz_t_1_x_1.5_div_sup100_sig_0_'+str(mode)+'.svg'))

In [ ]:
for mode in [0,1]:
    flist = []
    for f in [247, 241, 253]:
        flist.append(os.path.join(EFOLDERS[0], 'nb'+str(f)))
    fig = linePlots(flist, intmp.inkfunc, 1, 2.25, mode)
    fig.savefig(os.path.join(EXPORTFOLDER, 'HBnewt_velz_t_1_x_1.5_inkv_sup100_sig_0_'+str(mode)+'.svg'))

In [ ]:
for mode in [0,1]:
    flist = []
    for f in [221, 227, 709]:
        flist.append(os.path.join(EFOLDERS[1], 'nb'+str(f)))
    fig = linePlots(flist, intmp.supfunc, 1, 2.25, mode)
    fig.savefig(os.path.join(EXPORTFOLDER, 'newtHB_velz_t_1_x_1.5_supv_ink10_sig_0_'+str(mode)+'.svg'))

In [ ]:
flist = []
#for f in [32, 42, 52, 62]:
for f in [40,50,60]:
    flist.append(os.path.join(TOPFOLDER, 'nb'+str(f)))
fig = lineplots(flist, intmp.supfunc, 5, 2.25)
fig.savefig(os.path.join(EXPORTFOLDER, 'velz_t_5_sup_div0.1_sig_40.svg'))


In [ ]:
flist = []
for f in [22, 40, 41]:
    flist.append(os.path.join(TOPFOLDER, 'nb'+str(f)))
fig = linePlots(flist, intmp.sigfunc, 5, 2.25)
fig.saveFig(os.path.join(EXPORTFOLDER, 'velz_t_5_sig_div0.1_sup_1.svg'))

In [ ]:
# flist = []
# for f in [50, 42, 62]:
#     flist.append(os.path.join(TOPFOLDER, 'nb'+str(f)))
flist = [os.path.join(EFOLDERS[2], 'nb37'), os.path.join(EFOLDERS[0], 'nb247')]
for f in flist:
    stabilityPlot(f, EXPORTFOLDER, 2.5, 3, export=True)

In [ ]:
stabilityPlot(os.path.join(EFOLDERS[2], 'nb37'), EXPORTFOLDER, 2.5, 3, export=True)

In [ ]:
t = 0
for d in EFOLDERS:
    for f in os.listdir(d):
        l = intm.importLegend(os.path.join(d, f))
        if not type(l) is int:
            t1 = float(l.loc[5, 'val'])
            if t1>0:
                t+=t1
t

In [ ]:
import pandas as pd
metrics = []
for TOPFOLDER in EFOLDERS:
    for f in os.listdir(TOPFOLDER):
        try:
            m = metricVals(os.path.join(TOPFOLDER, f), 2.5, 5, ['arean', 'vertdispn', 'aspectratio', 'speeddecay'])
        except:
            pass
        else:
            metrics.append(m)
df = pd.DataFrame(metrics)
df

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3,sharex='col', sharey='row',figsize=(6.5,6.5))
cmap = plt.get_cmap('tab20c')

for t in range(len(EFOLDERS)):
    TOPFOLDER = EFOLDERS[t]
    metrics = []
    for f in os.listdir(TOPFOLDER):
        try:
            m = metricVals(os.path.join(TOPFOLDER, f), 2.5, 5, ['arean', 'vertdispn', 'aspectratio', 'speeddecay'])
        except:
            pass
        else:
            metrics.append(m)
    df = pd.DataFrame(metrics)

    df2 = df[df['arean']<3]
    df2 = df2[df2['vertdispn']>-0.1]
    df2 = df2[df2['vertdispn']<1.2]
    df2 = df2[df2['speeddecay']<1.1]

    slist = df.keys()
    for i in range(4):
        for j in range(i):
            color = cmap(t/3)
            if j==0 and i==1:
                axs[j,i-1].scatter(df2[slist[i]], df2[slist[j]], s=5, color=color, label=os.path.basename(TOPFOLDER))
            else:
                axs[j,i-1].scatter(df2[slist[i]], df2[slist[j]], s=5, color=color)
            axs[j,i-1].set_xlabel(slist[i])
            axs[j,i-1].set_ylabel(slist[j])
axs[0,0].legend()


In [ ]:
intm.exportIm(os.path.join(EXPORTFOLDER, 'correlations'), fig)